In [1]:
!pip install mlflow


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost
import pandas as pd
import numpy as np

In [3]:
df=pd.read_csv(r"C:\Users\kumar\Desktop\FAANG_PROJECT\FAANG - FAANG5.csv")
df

,Open,High,Low,Close,Volume,Market Cap,PE Ratio,EPS,Debt to Equity,Price to Book Ratio,Apple,Facebook,Google,Amazon,Netflix
0,1.156786,1.162679,1.117857,1.130179,293751500,3,2,3,0,1,0,1,0,0,0
1,1.139107,1.169107,1.124464,1.141786,293751500,3,2,3,0,1,0,1,0,0,0
2,1.151071,1.165179,1.143750,1.151786,293751500,3,2,3,0,1,0,1,0,0,0
3,1.154821,1.159107,1.130893,1.152679,293751500,3,2,3,0,1,0,1,0,0,0
4,1.160714,1.243393,1.156250,1.236607,293751500,3,2,3,0,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23050,319.531527,323.578279,316.243249,319.843610,2299500,4,3,0,3,0,0,0,0,0,1
23051,319.531527,323.578279,316.243249,319.843610,2797100,4,3,0,3,0,0,0,0,0,1
23052,319.531527,323.578279,316.243249,319.843610,2932800,4,3,0,3,0,0,0,0,0,1
23053,319.531527,323.578279,316.243249,319.843610,2494300,4,3,0,3,0,0,0,0,0,1


In [4]:
# Test-Train Split
from sklearn.model_selection import train_test_split

X = df.drop('Close', axis=1)
y = df['Close']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# Standard Scaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
!pip install xgboost



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error

In [8]:
models = [
("Linear Regressor", LinearRegression()),
("Decision Tree Regressor", DecisionTreeRegressor()),
("Random Forest Regressor", RandomForestRegressor()),
("XGBoost Regressor", XGBRegressor())
]

In [9]:
reports = []

for name, model in models:
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = mean_squared_error(y_test, y_pred)
    mae = mean_absolute_error(y_test, y_pred)
    reports.append((name, model, rmse, mae, r2))   

In [10]:
for name, model, rmse, mae, r2 in reports:
    print(f"Model: {name}")
    print(f"RMSE: {rmse}")
    print(f"MAE: {mae}")
    print(f"R2: {r2}")
    print("\n")

Model: Linear Regressor
RMSE: 0.5842477661994834
MAE: 0.35700672102228503
R2: 0.9999327529095073


Model: Decision Tree Regressor
RMSE: 1.2084889657832205
MAE: 0.4921546083553731
R2: 0.9998609025630171


Model: Random Forest Regressor
RMSE: 0.7613689815683902
MAE: 0.39158466778811934
R2: 0.9999123662052919


Model: XGBoost Regressor
RMSE: 1.2047963602788145
MAE: 0.5253679162668139
R2: 0.9998613275830015




In [11]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")
mlflow.set_experiment("FAANG Stock Closing Price Prediction")

for name, model, rmse, mae, r2 in reports:
    with mlflow.start_run(run_name=name) as run:
        mlflow.log_metric("RMSE", rmse)
        mlflow.log_metric("MAE", mae)
        mlflow.log_metric("R2", r2)
        
        if name == "Linear Regressor":
            mlflow.sklearn.log_model(model, "LR_model")
        elif name == "Decision Tree Regressor":
            mlflow.sklearn.log_model(model, "DT_model")
        elif name == "Random Forest Regressor":
            mlflow.sklearn.log_model(model, "RF_model")
        elif name == "XGBoost Regressor":
            mlflow.xgboost.log_model(model, "XGB_model")

2025/03/16 14:17:14 INFO mlflow.tracking.fluent: Experiment with name 'FAANG Stock Closing Price Prediction' does not exist. Creating a new experiment.
2025/03/16 14:17:14 WARNING mlflow.utils.git_utils: Failed to import Git (the Git executable is probably not on your PATH), so Git SHA is not available. Error: Failed to initialize: Bad git executable.
The git executable must be specified in one of the following ways:
    - be included in your $PATH
    - be set via $GIT_PYTHON_GIT_EXECUTABLE
    - explicitly set via git.refresh(<full-path-to-git-executable>)

All git commands will error until this is rectified.

This initial message can be silenced or aggravated in the future by setting the
$GIT_PYTHON_REFRESH environment variable. Use one of the following values:
    - quiet|q|silence|s|silent|none|n|0: for no message or exception
    - warn|w|warning|log|l|1: for a warning message (logging level CRITICAL, displayed by default)
    - error|e|exception|raise|r|2: for a raised exception

🏃 View run Linear Regressor at: http://127.0.0.1:5000/#/experiments/202491072935989772/runs/8d77bdd794ef4cb88ae40402ac19e005
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/202491072935989772


2025/03/16 14:17:31 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Decision Tree Regressor at: http://127.0.0.1:5000/#/experiments/202491072935989772/runs/149284c96a814e5aa92f30154afb1305
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/202491072935989772


2025/03/16 14:17:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest Regressor at: http://127.0.0.1:5000/#/experiments/202491072935989772/runs/8ed1bc6e59e349fb9627a2602ed2fac1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/202491072935989772


c:\Users\kumar\Desktop\FAANG_PROJECT\faang\Lib\site-packages\xgboost\core.py:158: UserWarning: [14:17:43] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)
2025/03/16 14:17:52 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBoost Regressor at: http://127.0.0.1:5000/#/experiments/202491072935989772/runs/6f5660453478466db53ec04ca40c5187
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/202491072935989772


In [16]:
model_uri

'runs:/4e819e8c78184348af5886df86acceba/RF_model'

In [18]:
import mlflow

experiment_id = "202491072935989772"  # Replace with your actual experiment ID
runs = mlflow.search_runs(experiment_ids=[experiment_id])
print(runs[['run_id', 'status']])


                             run_id    status
0  6f5660453478466db53ec04ca40c5187  FINISHED
1  8ed1bc6e59e349fb9627a2602ed2fac1  FINISHED
2  149284c96a814e5aa92f30154afb1305  FINISHED
3  8d77bdd794ef4cb88ae40402ac19e005  FINISHED


In [19]:
model_name ='Random Forest Regressor'
run_id = '8ed1bc6e59e349fb9627a2602ed2fac1'
model_uri = f'runs:/{run_id}/RF_model'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri= model_uri , name= model_name)

Successfully registered model 'Random Forest Regressor'.
2025/03/16 14:38:34 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: Random Forest Regressor, version 1


🏃 View run Random Forest Regressor at: http://127.0.0.1:5000/#/experiments/202491072935989772/runs/8ed1bc6e59e349fb9627a2602ed2fac1
🧪 View experiment at: http://127.0.0.1:5000/#/experiments/202491072935989772


Created version '1' of model 'Random Forest Regressor'.


In [20]:
import pickle

with open("standard_scaler.pkl", "wb") as s:
    pickle.dump(scaler, s)